###  Apply transfer learning to reduce the dependence on data, given the limited amount of available data.
1. VGGFACE pretrained models: VGG16, ResNet50, SeNet
2. Data Preprocessing: Face Extractions by YOLO
3. Data Augmentation: Rotations, Flips
4. EarlyStopping Applied to prevent the model from overfitting in the training process during training process

##### 1. Import APIs
tensorflow 1.7+

In [1]:
import tensorflow as tf
import keras
import warnings

# Filter out FutureWarnings
warnings.filterwarnings('ignore', category=FutureWarning)

print("tensorflow version: ", tf.__version__)
from keras_vggface.vggface import VGGFace
import dlib
from sklearn.model_selection import KFold
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from keras.preprocessing import image
import matplotlib.pyplot as plt
import os
import random

from models.custom_model import get_custom_model
import numpy as np
from keras.preprocessing import image
from keras_vggface.vggface import VGGFace
from keras_vggface import utils

from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, BatchNormalization, Dropout
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support




/home/larry5/.conda/envs/yoloclone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/larry5/.conda/envs/yoloclone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/larry5/.conda/envs/yoloclone/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/larry5/.conda/envs/yoloclone/lib/python3

tensorflow version:  1.7.0


####  Testing

##### 1. Define the Data Generator

In [2]:
# activate the rotation and horizontal flip
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
#     validation_split = 0.2,
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True)
valid_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
#     validation_split = 0.2,
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True)
test_datagen  = ImageDataGenerator(rescale = 1./255)

In [3]:
# Get the model
custom_pretrained_model = get_custom_model()

# Compile, fit, or evaluate the model
# For example, to compile:
custom_pretrained_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Now you can fit the model to your data, evaluate it, etc.

testdata = test_datagen.flow_from_directory(directory="../../../neo/PainDetector/data/face_dir_final/",target_size=(224,224), class_mode='categorical',shuffle= False,batch_size=8)


/home/larry5/.conda/envs/yoloclone/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Found 187 images belonging to 2 classes.


In [4]:
GT = []
pre = np.array([])

weight_address = "../../data/weights/model_weight.hdf5"


try:
    custom_pretrained_model.load_weights(weight_address)
    print("Weights loaded...")
except:
    pass

#obtian results
pred=custom_pretrained_model.predict_generator(testdata, steps=len(testdata), verbose=1)
filenames=testdata.filenames
for i in filenames:  #load gt label
    if "no" in i.split('/')[0]:
        GT.append(0)
    else:
        GT.append(1)
gt = np.array(GT)  #convert to np array

predicted_class_indices=np.argmax(pred,axis=1)  #obtain predictions


cnf_matrix = confusion_matrix(gt, predicted_class_indices)  
cm = cnf_matrix
TN  = cm[0][0]  
FN  = cm[1][0]  
TP  = cm[1][1] 
FP  = cm[0][1] 
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)
# #  true positive rate
# TPR = TP/(TP+FN)
# # true negative rate
# TNR = TN/(TN+FP) 
# # false positive rate
# FPR = FP/(FP+TP)
# # False negative rate
# FNR = FN/(TN+FN)
print("Calculate accuracy ...")
single_branch_ACC =  (TP + TN)/(FP+TN + FN +TP)
print("Accuracy reslut:", single_branch_ACC)

# threshold 
k = 0.55 
small_index = np.where( (pred[:,1] <= k) & (pred[:,1] > (1-k)))  #check
predicted_class_indices[small_index] = gt[small_index]  #update
print("Num of cases sent to manual branch:", small_index[0].shape[0])
num_cor = np.sum(gt == predicted_class_indices) #TP+TN
print("Final accuracy: ", num_cor / gt.shape)

Weights loaded...
24/24 [==============================] - 9s 364ms/step
Calculate accuracy ...
Accuracy reslut: 0.7754010695187166
Num of cases sent to manual branch: 10
Final accuracy:  [0.82352941]


#### Training

##### 1. Hyperparameters 

In [ ]:
#custom parameters
learning_rate = 0.00001
nb_class = 2
hidden_dim = 512
batch_size = 32
img_height = 224
img_width = 224
data_dir = "../../data/COPE/"
print("Does the data folder exist: ", os.path.isdir(data_dir))

##### 2. Train the model

In [ ]:
custom_pretrained_model = get_custom_model()
opt = Adam(lr=learning_rate)
custom_pretrained_model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
path = "../../data/weights/" + "weights-{epoch:03d}-{val_acc:.4f}.hdf5"


traindata = train_datagen.flow_from_directory(directory=data_dir  +"/training/",target_size=(224,224),class_mode='categorical',shuffle= True,batch_size=8)
testdata = valid_datagen.flow_from_directory(directory=data_dir  +"/validation/",target_size=(224,224),class_mode='categorical',shuffle= True,batch_size=8)
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = custom_pretrained_model.fit_generator(generator=traindata, validation_data= testdata, epochs=100,callbacks=[checkpoint,early])
